# Training SimpleNN on CIFAR-10
In this project, you will use the SimpleNN model to perform image classification on CIFAR-10. CIFAR-10 orginally contains 60K images from 10 categories. We split it into 45K/5K/10K images to serve as train/valiation/test set. We only release the ground-truth labels of training/validation dataset to you.

## Step 0: Set up the SimpleNN model
As you have practiced to implement simple neural networks in Homework 1, we just prepare the implementation for you.

In [48]:
# import necessary dependencies
import argparse
import os, sys
import time
import datetime
from tqdm import tqdm_notebook as tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F

In [49]:
# define the SimpleNN mode;
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 8, 5)
        self.conv2 = nn.Conv2d(8, 16, 3)
        self.fc1   = nn.Linear(16*6*6, 120)
        self.fc2   = nn.Linear(120, 84)
        self.fc3   = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

### Question (a)
Here is a sanity check to verify the implementation of SimpleNN. 
You need to:
1. Write down your code.
2. **In the PDF report**, give a brief description on how the code helps you know that SimpleNN is implemented correctly.

In [50]:
#############################################
# your code here
# sanity check for the correctness of SimpleNN
model = SimpleNN()

dummy_input = torch.randn(4,3,32,32)
output = model(dummy_input)

print(output.shape)

# 

#############################################

torch.Size([4, 10])


## Step 1: Set up preprocessing functions
Preprocessing is very important as discussed in the lecture.
You will need to write preprocessing functions with the help of *torchvision.transforms* in this step.
You can find helpful tutorial/API at [here](https://pytorch.org/vision/stable/transforms.html).

### Question (b)
For the question, you need to:
1. Complete the preprocessing code below.
2. **In the PDF report**, briefly describe what preprocessing operations you used and what are the purposes of them.

Hint: 
1. Only two operations are necessary to complete the basic preprocessing here.
2. The raw input read from the dataset will be PIL images.
3. Data augmentation operations are not mendatory, but feel free to incorporate them if you want.
4. Reference value for mean/std of CIFAR-10 images (assuming the pixel values are within [0,1]): mean (RGB-format): (0.4914, 0.4822, 0.4465), std (RGB-format): (0.2023, 0.1994, 0.2010)

In [51]:
# useful libraries
import torchvision
import torchvision.transforms as transforms

#############################################
# your code here
# specify preprocessing function

# Define data augmentation and preprocessing for training data
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], 
                         std=[0.2023, 0.1994, 0.2010])
])

# Define preprocessing for validation data (no augmentation)
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465],
                         std=[0.2023, 0.1994, 0.2010])
])
#############################################

## Step 2: Set up dataset and dataloader

### Question (c)
Set up the train/val datasets and dataloders that are to be used during the training. Check out the [official API](https://pytorch.org/docs/stable/data.html) for more information about **torch.utils.data.DataLoader**.

Here, you need to:
1. Complete the code below.

In [52]:
# do NOT change these
from tools.dataset import CIFAR10
from torch.utils.data import DataLoader

# a few arguments, do NOT change these
DATA_ROOT = "./data"
TRAIN_BATCH_SIZE = 128
VAL_BATCH_SIZE = 100

#############################################
# your code here
# construct dataset
train_set = CIFAR10(
    root=DATA_ROOT, 
    mode='train', 
    download=True,
    transform=transform_train   # your code
)
val_set = CIFAR10(
    root=DATA_ROOT, 
    mode='val', 
    download=True,
    transform=transform_val # your code
)

# construct dataloader
train_loader = DataLoader(
    train_set, 
    batch_size=TRAIN_BATCH_SIZE ,  # your code
    shuffle= True,     # your code
    num_workers=4
)
val_loader = DataLoader(
    val_set, 
    batch_size=VAL_BATCH_SIZE,  # your code
    shuffle=False,     # your code
    num_workers=4
)
#############################################

Using downloaded and verified file: ./data\cifar10_trainval_F22.zip
Extracting ./data\cifar10_trainval_F22.zip to ./data
Files already downloaded and verified
Using downloaded and verified file: ./data\cifar10_trainval_F22.zip
Extracting ./data\cifar10_trainval_F22.zip to ./data
Files already downloaded and verified


## Step 3: Instantiate your SimpleNN model and deploy it to GPU devices.
### Question (d)
You may want to deploy your model to GPU device for efficient training. Please assign your model to GPU if possible. If you are training on a machine without GPUs, please deploy your model to CPUs.

Here, you need to:
1. Complete the code below.
2. **In the PDF report**, briefly describe how you verify that your model is indeed deployed on GPU. (Hint: check $\texttt{nvidia-smi}$.)

In [53]:
# specify the device for computation
#############################################
# your code here
device = torch.device("cuda")

model.to(device)
print(device)
!nvidia-smi
#############################################

cuda
Wed Feb 19 23:38:38 2025       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 512.77       Driver Version: 512.77       CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   36C    P8    25W / 320W |   3018MiB / 10240MiB |     15%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

## Step 4: Set up the loss function and optimizer
Loss function/objective function is used to provide "feedback" for the neural networks. Typically, we use multi-class cross-entropy as the loss function for classification models. As for the optimizer, we will use SGD with momentum. 

### Question (e)
Here, you need to:
1. Set up the cross-entropy loss as the criterion. (Hint: there are implemented functions in **torch.nn**)
2. Specify a SGD optimizer with momentum. (Hint: there are implemented functions in **torch.optim**)

In [54]:
import torch.nn as nn
import torch.optim as optim

# hyperparameters, do NOT change right now
# initial learning rate
INITIAL_LR = 0.01

# momentum for optimizer
MOMENTUM = 0.9

# L2 regularization strength
REG = 1e-4

#############################################
# your code here
# create loss function
criterion = nn.CrossEntropyLoss().to(device)

# Add optimizer
optimizer = optim.SGD(model.parameters(), lr=INITIAL_LR, momentum=MOMENTUM, weight_decay=REG)

## Step 5: Start the training process.

### Question (f)/(g)
Congratulations! You have completed all of the previous steps and it is time to train our neural network.

Here you need to:
1. Complete the training codes.
2. Actually perform the training.

Hint: Training a neural network usually repeats the following 4 steps: 

**i) Get a batch of data from the dataloader and copy it to your device (GPU).**

**ii) Do a forward pass to get the outputs from the neural network and compute the loss. Be careful about your inputs to the loss function. Are the inputs required to be the logits or softmax probabilities?)**

**iii) Do a backward pass (back-propagation) to compute gradients of all weights with respect to the loss.**

**iiii) Update the model weights with the optimizer.**

You will also need to compute the accuracy of training/validation samples to track your model's performance over each epoch (the accuracy should be increasing as you train for more and more epochs).


In [55]:
# some hyperparameters
# total number of training epochs
EPOCHS = 30

# the folder where the trained model is saved
CHECKPOINT_FOLDER = "./saved_model"

# start the training/validation process
# the process should take about 5 minutes on a GTX 1070-Ti
# if the code is written efficiently.
best_val_acc = 0
current_learning_rate = INITIAL_LR

print("==> Training starts!")
print("="*50)

l_train_acc = []
l_train_loss = []
l_val_acc = []
l_val_loss = []

for i in range(0, EPOCHS):    
    #######################
    # your code here
    # switch to train mode
    model.train()
    
    #######################
    
    print("Epoch %d:" %i)
    # this help you compute the training accuracy
    total_examples = 0
    correct_examples = 0

    train_loss = 0 # track training loss if you want
    
    # Train the model for 1 epoch.
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        ####################################
        # your code here
        # copy inputs to device
        inputs, targets = inputs.to(device), targets.to(device)
        
        # compute the output and loss
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # zero the gradient
        optimizer.zero_grad()
        
        # backpropagation
        loss.backward()
        
        # apply gradient and update the weights
        optimizer.step()
        
        # count the number of correctly predicted samples in the current batch
        _, predicted = outputs.max(1)

        total_examples += targets.size(0)
        correct_examples += predicted.eq(targets).sum().item()
        
        train_loss += loss.item()
        ####################################
                
    avg_loss = train_loss / len(train_loader)
    avg_acc = correct_examples / total_examples
    l_train_acc.append(avg_acc)
    l_train_loss.append(avg_loss)
    print("Training loss: %.4f, Training accuracy: %.4f" %(avg_loss, avg_acc))

    # Validate on the validation dataset
    #######################
    # your code here
    # switch to eval mode
    
    model.eval()
    #######################

    # this help you compute the validation accuracy
    total_examples = 0
    correct_examples = 0
    
    val_loss = 0 # again, track the validation loss if you want

    # disable gradient during validation, which can save GPU memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            ####################################
            # your code here
            # copy inputs to device
            inputs, targets = inputs.to(device), targets.to(device)
            
            # compute the output and loss
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            # count the number of correctly predicted samples in the current batch
            _, predicted = outputs.max(1)
            total_examples += targets.size(0)
            correct_examples += predicted.eq(targets).sum().item()
            
            val_loss += loss.item() 
            ####################################

    avg_loss = val_loss / len(val_loader)
    avg_acc = correct_examples / total_examples
    l_val_loss.append(avg_loss)
    l_val_acc.append(avg_acc)
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_loss, avg_acc))
    
    # save the model checkpoint
    if avg_acc > best_val_acc:
        best_val_acc = avg_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
           os.makedirs(CHECKPOINT_FOLDER)
        print("Saving ...")
        state = {'state_dict': model.state_dict(),
                'epoch': i,
                'lr': current_learning_rate}
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
        
    print('')

print("="*50)
print(f"==> Optimization finished! Best validation accuracy: {best_val_acc:.4f}")

==> Training starts!
Epoch 0:
Training loss: 1.8819, Training accuracy: 0.3108
Validation loss: 1.5710, Validation accuracy: 0.4330
Saving ...

Epoch 1:
Training loss: 1.4218, Training accuracy: 0.4838
Validation loss: 1.3844, Validation accuracy: 0.5054
Saving ...

Epoch 2:
Training loss: 1.2449, Training accuracy: 0.5564
Validation loss: 1.2390, Validation accuracy: 0.5654
Saving ...

Epoch 3:
Training loss: 1.1272, Training accuracy: 0.6053
Validation loss: 1.2017, Validation accuracy: 0.5862
Saving ...

Epoch 4:
Training loss: 1.0305, Training accuracy: 0.6402
Validation loss: 1.1099, Validation accuracy: 0.6176
Saving ...

Epoch 5:
Training loss: 0.9556, Training accuracy: 0.6655
Validation loss: 1.0540, Validation accuracy: 0.6386
Saving ...

Epoch 6:
Training loss: 0.8941, Training accuracy: 0.6899
Validation loss: 1.1218, Validation accuracy: 0.6282

Epoch 7:
Training loss: 0.8449, Training accuracy: 0.7036
Validation loss: 1.0760, Validation accuracy: 0.6326

Epoch 8:
Training

In [56]:
# visualize the training process
import matplotlib.pyplot as plt
%config InlineBackend.figure_format = 'retina' 

plt.figure(figsize=(7,5), dpi= 100) 
plt.plot(l_train_acc, label='train set')
plt.plot(l_val_acc, label='val set')
plt.text(s=f"Best val acc: {best_val_acc:.4f}", x=20, y=0.3, fontsize=12)
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy vs. Epochs')
plt.tight_layout()
plt.show()

# Bonus: with learning rate decay

The following code can help you adjust the learning rate during training. You need to figure out how to incorporate this code into your training loop.
```python
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate = current_learning_rate * DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" %current_learning_rate)
```

In [57]:
# --- Hyperparameters ---
EPOCHS = 30
CHECKPOINT_FOLDER = "./saved_model"

# Learning rate decay settings for the bonus question:
DECAY_EPOCHS = 10  # Decay learning rate every 10 epochs
DECAY = 0.5        # Multiply learning rate by 0.5 each decay step

# Initial learning rate
current_learning_rate = INITIAL_LR

best_val_acc = 0

print("==> Training starts!")
print("=" * 50)

for i in range(0, EPOCHS):
    
    # --- Learning Rate Decay Step ---
    # Update learning rate after every DECAY_EPOCHS (except for the 0th epoch)
    if i % DECAY_EPOCHS == 0 and i != 0:
        current_learning_rate *= DECAY
        for param_group in optimizer.param_groups:
            param_group['lr'] = current_learning_rate
        print("Current learning rate has decayed to %f" % current_learning_rate)
    
    # --- Training Phase ---
    model.train()
    print("Epoch %d:" % i)
    
    total_examples = 0
    correct_examples = 0
    train_loss = 0  # Accumulate training loss
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        # Move inputs and targets to device (GPU/CPU)
        inputs, targets = inputs.to(device), targets.to(device)
        
        # Compute outputs and loss
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        
        # Zero the gradients, backpropagate, and update weights
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Update running totals for loss and accuracy
        _, predicted = outputs.max(1)
        total_examples += targets.size(0)
        correct_examples += predicted.eq(targets).sum().item()
        train_loss += loss.item()
    
    # Compute average loss and accuracy for the epoch
    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = correct_examples / total_examples
    print("Training loss: %.4f, Training accuracy: %.4f" % (avg_train_loss, avg_train_acc))
    
    # --- Validation Phase ---
    model.eval()
    total_examples = 0
    correct_examples = 0
    val_loss = 0
    
    # Disable gradient computation for validation to save memory
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            _, predicted = outputs.max(1)
            total_examples += targets.size(0)
            correct_examples += predicted.eq(targets).sum().item()
            val_loss += loss.item()
    
    avg_val_loss = val_loss / len(val_loader)
    avg_val_acc = correct_examples / total_examples
    print("Validation loss: %.4f, Validation accuracy: %.4f" % (avg_val_loss, avg_val_acc))
    
    # --- Save the Model Checkpoint ---
    if avg_val_acc > best_val_acc:
        best_val_acc = avg_val_acc
        if not os.path.exists(CHECKPOINT_FOLDER):
            os.makedirs(CHECKPOINT_FOLDER)
        print("Saving model checkpoint...")
        state = {
            'state_dict': model.state_dict(),
            'epoch': i,
            'lr': current_learning_rate
        }
        torch.save(state, os.path.join(CHECKPOINT_FOLDER, 'simplenn.pth'))
    
    print('')

print("=" * 50)
print("==> Optimization finished! Best validation accuracy: %.4f" % best_val_acc)


==> Training starts!
Epoch 0:
Training loss: 0.2664, Training accuracy: 0.9052
Validation loss: 2.1200, Validation accuracy: 0.6116
Saving model checkpoint...

Epoch 1:
Training loss: 0.2734, Training accuracy: 0.9039
Validation loss: 2.0019, Validation accuracy: 0.6210
Saving model checkpoint...

Epoch 2:
Training loss: 0.2513, Training accuracy: 0.9111
Validation loss: 2.0425, Validation accuracy: 0.6230
Saving model checkpoint...

Epoch 3:
Training loss: 0.2452, Training accuracy: 0.9117
Validation loss: 2.1705, Validation accuracy: 0.6162

Epoch 4:
Training loss: 0.2430, Training accuracy: 0.9136
Validation loss: 2.2024, Validation accuracy: 0.6162

Epoch 5:
Training loss: 0.2334, Training accuracy: 0.9176
Validation loss: 2.2524, Validation accuracy: 0.6138

Epoch 6:
Training loss: 0.2297, Training accuracy: 0.9185
Validation loss: 2.3651, Validation accuracy: 0.6178

Epoch 7:
Training loss: 0.2210, Training accuracy: 0.9220
Validation loss: 2.3863, Validation accuracy: 0.6202

Ep

In [58]:
# visualiztion of training process with learning rate decay 
plt.figure(figsize=(7,5), dpi= 100) 
plt.plot(l_train_acc, label='train set')
plt.plot(l_val_acc, label='val set')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.text(s=f"Best val acc: {best_val_acc:.4f}", x=20, y=0.3, fontsize=12)
plt.legend()
plt.title('Accuracy vs. Epochs with rate decay')
plt.tight_layout()
plt.show()